In [22]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM,Dense,Flatten
from keras.preprocessing.text import *
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.utils import to_categorical
from pickle import load
from pickle import dump
from keras.models import load_model
import string
from numpy import array
from random import randint

In [23]:
f = open('path_to_\Bible_for_word.txt','r')
lines = f.readlines()

In [25]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
print(sequences[0])
print(sequences[1])
sequences = pad_sequences(sequences, maxlen=15, padding='post')
print(sequences[0])
print(sequences[1])

[6, 1, 669, 26, 1268, 1, 171, 2, 1, 109, 2, 1, 109, 27, 222, 1972, 2, 1973, 2, 482, 27, 44, 1, 227, 3, 1, 956, 2, 1, 187, 3, 26, 852, 44, 1, 227, 3, 1, 303]
[2, 26, 36, 79, 53, 16, 320, 2, 53, 27, 320]
[  3   1 956   2   1 187   3  26 852  44   1 227   3   1 303]
[  2  26  36  79  53  16 320   2  53  27 320   0   0   0   0]


In [26]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

12606


In [27]:
# separate into input and output
sequences = np.array(sequences)
print(sequences.shape)
X, y = np.array(sequences[:,:-1]), np.array(sequences[:,-1])
print(X[0])
y = y.reshape(30382,1)
print(y[0])
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]
print(X.shape)
print(y.shape)

(30382, 15)
[  3   1 956   2   1 187   3  26 852  44   1 227   3   1]
[303]
(30382, 14)
(30382, 12606)


In [28]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=seq_length))
model.add(LSTM(100, return_sequences=True)) 
model.add(LSTM(100)) 
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 14, 100)           1260600   
_________________________________________________________________
lstm_3 (LSTM)                (None, 14, 100)           80400     
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 12606)             1273206   
Total params: 2,704,706
Trainable params: 2,704,706
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=50)

# save the model to file
model.save('path_to_\model.h5')
# save the tokenizer
dump(tokenizer, open('path_to_\tokenizer.pkl', 'wb'))

In [20]:

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
 
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)
 
# load cleaned text sequences
in_filename = 'path_to_\Bible_for_word.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
#seq_length = len(lines[0].split()) - 1
seq_length = 14 
# load the model
model = load_model('path_to_\model.h5')
 
# load the tokenizer
tokenizer = load(open('path_to_\tokenizer.pkl', 'rb'))
 
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
 
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated) # the output below is due to 50 epochs and accuracy = 60

For the earth bringeth forth fruit of herself first the blade then the ear after that the full corn in the ear 

side them out away up it up it up it up it out it out it up it out it out it up it out it out it up it out it out it up it out it out it up it out it out it up it out it
